# Imports

In [2]:
import time
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime

warnings.filterwarnings('ignore')

# Constant

In [3]:
ARIMA_PREDICTIONS = "/kaggle/input/weather-underground/ARIMA_Forecasts_Datetime.csv"
GM_PREDICTIONS = "/kaggle/input/weather-underground/GM_Forecasts.csv"

# Code

In [6]:
df_arima_predictions = pd.read_csv(ARIMA_PREDICTIONS)
df_gm_predictions = pd.read_csv(GM_PREDICTIONS)

In [7]:
df_arima_predictions.head(3)

,Index,Forecasts
0,8321,-0.487280
1,8322,-0.318901
2,8323,-0.252342


In [8]:
df_gm_predictions.head(3)

,Index,Datetime,Actual,Forecasts
0,8321,2024-05-25 00:30:00,0.0,25.905326
1,8322,2024-05-25 01:00:00,0.0,25.907901
2,8323,2024-05-25 01:30:00,0.0,25.910475
